Code to train the network

In [1]:
num_actions = 4

In [2]:
import numpy as np
import tensorflow as tf

/Users/adam/anaconda3/envs/energy_py/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/adam/anaconda3/envs/energy_py/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
num_actions = 5

In [4]:
q_vals = tf.placeholder(shape=(None, num_actions), dtype=tf.float32)

selected_action = tf.placeholder(shape=(None), dtype=tf.int64)

q_selected_action = tf.reduce_sum(q_vals * tf.one_hot(selected_action, num_actions), 1)

q_selected_action = tf.reduce_sum(q_vals * tf.squeeze(tf.one_hot(selected_action, num_actions)), 1)

# . TODO compare with and without squeeze

In [5]:
qv = np.arange(15).reshape(3, -1)
a = np.random.randint(low=0, high=num_actions-1, size=(3,1))

with tf.Session() as sess:
    q_vals = sess.run(q_selected_action,
                   {q_vals: qv,
                    selected_action: a})

In [6]:
qv

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [7]:
a

array([[1],
       [2],
       [0]])

In [33]:
q_vals

array([ 1.,  7., 10.], dtype=float32)

# Masking q values

In [26]:
cond = tf.placeholder(shape=(None), dtype=tf.bool)
i = tf.placeholder(shape=(None), dtype=tf.float32)  

zeros = tf.zeros_like(i)
new_tensor = tf.where(condition=cond, x=i, y=zeros)

In [30]:
with tf.Session() as sess:
    o = sess.run(new_tensor,
                {cond: np.array([True, False, True]),
                 i: q_vals})

In [31]:
o

array([ 1.,  0., 10.], dtype=float32)

In [ ]:
# . compare this with the baselines approach for speed